In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from pathlib import Path


In [3]:
datasets, info = tfds.load(name = 'mnist', with_info = True, as_supervised= True,data_dir = './data')
mnist_train, mnist_test = datasets['train'], datasets['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
strategy = tf.distribute.MirroredStrategy()
print(f'Number of devices = {strategy.num_replicas_in_sync}')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices = 1


In [6]:
train_example_num = info.splits['train'].num_examples
test_example_num = info.splits['test'].num_examples

In [7]:
buffer_size = 10000
batch_size_per_replica = 64
batch_size = batch_size_per_replica*strategy.num_replicas_in_sync

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image/= 255.0
    return image, label

In [9]:
trn_dataset = mnist_train.map(scale).cache().shuffle(buffer_size).batch(batch_size)
tst_dataset = mnist_test.map(scale).batch(batch_size)

In [10]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32,3, activation = 'relu', input_shape= (28,28,1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10)
    ])
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
             optimizer= tf.keras.optimizers.Adam(),
             metrics=['accuracy']) 

In [11]:
model.fit(trn_dataset,epochs = 12)

Epoch 1/12
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


938/938 [==============================] - 5s 6ms/step - loss: 0.2001 - accuracy: 0.9413
Epoch 2/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0654 - accuracy: 0.9804
Epoch 3/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0459 - accuracy: 0.9861
Epoch 4/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0339 - accuracy: 0.9898
Epoch 5/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0252 - accuracy: 0.9921
Epoch 6/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0190 - accuracy: 0.9947
Epoch 7/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0150 - accuracy: 0.9955
Epoch 8/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0109 - accuracy: 0.9969
Epoch 9/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0096 - accuracy: 0.9969
Epoch 10/12
938/938 [==============================] - 5s 6ms/step - loss: 0.0070 - accuracy: 0.9980
Epoch 11/1